### Outline
 - Save information about the radio image files
 - Probe dendrogram values to find best source detection parameters
 - Use the dendrogram to save source regions to a catalog file
 - Reject bad sources
 - Match source detections between bands
 - Make flux measurements within several different aperture shapes
 - Flux flux plots
 - Flux histograms

### Image File Information
Here, I have already created a `.dat` file that contains information about each radio image file we'll be processing. Let's read it in and see what info we'll need from each file:

In [1]:
from astropy.table import Table

imgfileinfo = Table.read('.imgfileinfo.dat', format='ascii')
print(imgfileinfo.colnames)

['filename', 'region', 'band', 'min_value', 'delta_fraction', 'min_npix', 'nu', 'ppbeam']


The filenames are pretty long -- they've been hidden here so we can see what the rest of the table contains.

In [2]:
print(imgfileinfo[imgfileinfo.colnames[1:]])

region band min_value delta_fraction min_npix       nu             ppbeam     
------ ---- --------- -------------- -------- -------------- -----------------
 w51e2    3   0.00015            1.7      7.5 92982346121.92 125.0926865069147
 w51e2    6  0.000325            1.7      7.5 226092028953.4 101.7249966753436


 - Region: a string identifier for the field we're looking at
 - Band: the ALMA band number for the observation
 - Min value, delta fraction, min npix: default dendrogram parameters we'll get to later
 - Nu: the frequency of the observation in Hz, obtained from the FITS image header
 - ppbeam: pixels per beam, which we'll calculate later

## Probing Dendrogram Values

In [3]:
from probe import probe
import numpy as np

In [4]:
min_values = np.linspace(0.00015, 0.0003, 5)

Min value: the "detection level" for the dendrogram. We're going to test a range of values -- for each of these tests, any pixel with flux less than the detection level won't be included in the dendrogram.

In [5]:
delta_frac = 1.5
min_deltas = min_values*delta_frac

Min delta: the tolerance between one contour and the next. It's best to use min deltas proportional to the min value, so for convenience we'll store and use the value `delta_frac` instead.

In [6]:
min_npixs = [5, 7.5, 10]

Min npix: the minimum number of pixels within a region needed to qualify it as a "source".

In [ ]:
region = 'w51e2'
band = 3

In [7]:
probe(region, band, min_values, min_deltas, min_npixs)

Min values:  [0.00015   0.0001875 0.000225  0.0002625 0.0003   ]
Min deltas:  [0.000225   0.00028125 0.0003375  0.00039375 0.00045   ]
Min npix:  [5, 7.5, 10] 

Total task progress:


Min value: 0.00015    Min delta: 0.000225   Min npix: 5
Generating dendrogram using 336,503 of 216,090,000 pixels (0.15572354111712713% of data)

[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>                                        ] 0%
[>          

[=>                                       ] 4%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>                                      ] 5%
[==>         

[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                    ] 10%
[====>                                  

[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                  ] 15%
[======>                                

[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                              

...

[======================================>  ] 97%
[======================================>  ] 97%
[=======================================> ] 97%
[=======================================> ] 97%
[=======================================> ] 97%
[=======================================> ] 98%
[=======================================> ] 98%
[=======================================> ] 98%
[=======================================> ] 98%
[=======================================> ] 98%
[=======================================> ] 98%
[=======================================> ] 99%
[=======================================> ] 99%
[=======================================> ] 99%
[=======================================> ] 99%
[=======================================> ] 99%
[=======================================> ] 99%
[========================================>] 100%
Computing catalog for 45 structures

[>                                        ] 2%
[=>                                       ] 4%
[==>

[=======>                                 ] 17%
[=======>                                 ] 17%
[=======>                                 ] 17%
[=======>                                 ] 18%
[=======>                                 ] 18%
[=======>                                 ] 18%
[=======>                                 ] 18%
[=======>                                 ] 18%
[=======>                                 ] 18%
[=======>                                 ] 19%
[=======>                                 ] 19%
[=======>                                 ] 19%
[=======>                                 ] 19%
[=======>                                 ] 19%
[=======>                                 ] 19%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                                ] 20%
[========>                              

[==================>                      ] 46%
[==================>                      ] 46%
[==================>                      ] 46%
[==================>                      ] 46%
[==================>                      ] 47%
[==================>                      ] 47%
[==================>                      ] 47%
[===================>                     ] 47%
[===================>                     ] 47%
[===================>                     ] 47%
[===================>                     ] 48%
[===================>                     ] 48%
[===================>                     ] 48%
[===================>                     ] 48%
[===================>                     ] 48%
[===================>                     ] 48%
[===================>                     ] 49%
[===================>                     ] 49%
[===================>                     ] 49%
[===================>                     ] 49%
[===================>                   

[==============================>          ] 75%
[==============================>          ] 75%
[==============================>          ] 75%
[==============================>          ] 75%
[==============================>          ] 75%
[==============================>          ] 75%
[==============================>          ] 76%
[==============================>          ] 76%
[==============================>          ] 76%
[==============================>          ] 76%
[==============================>          ] 76%
[==============================>          ] 76%
[==============================>          ] 77%
[==============================>          ] 77%
[==============================>          ] 77%
[===============================>         ] 77%
[===============================>         ] 77%
[===============================>         ] 77%
[===============================>         ] 78%
[===============================>         ] 78%
[===============================>       

KeyboardInterrupt: 

Catalog and region files are output to `./cat/` and `./reg/`, respectively. Now, we can go into DS9 and see which parameters had the most success.

For band 3, the best parameters I found were